In [1]:
Dataset = open('../Dataset/Lichess_Trimmed.txt')

In [2]:
line = Dataset.readline()
line = line.strip()
print(line.split())

['1.', 'e4', '{', '[%eval', '0.36]', '[%clk', '0:07:00]', '}', '1...', 'c5', '{', '[%eval', '0.32]', '[%clk', '0:07:00]', '}', '2.', 'Nf3', '{', '[%eval', '0.0]', '[%clk', '0:06:59]', '}', '2...', 'Nc6', '{', '[%eval', '0.0]', '[%clk', '0:06:56]', '}', '3.', 'd4', '{', '[%eval', '0.33]', '[%clk', '0:06:59]', '}', '3...', 'd6?', '{', '[%eval', '1.41]', '[%clk', '0:06:50]', '}', '4.', 'dxc5', '{', '[%eval', '0.95]', '[%clk', '0:06:57]', '}', '4...', 'dxc5', '{', '[%eval', '1.31]', '[%clk', '0:06:49]', '}', '5.', 'Qxd8+', '{', '[%eval', '1.93]', '[%clk', '0:06:56]', '}', '5...', 'Nxd8', '{', '[%eval', '1.67]', '[%clk', '0:06:48]', '}', '6.', 'Bb5+', '{', '[%eval', '1.57]', '[%clk', '0:06:54]', '}', '6...', 'Bd7', '{', '[%eval', '1.66]', '[%clk', '0:06:45]', '}', '7.', 'Bxd7+', '{', '[%eval', '0.98]', '[%clk', '0:06:53]', '}', '7...', 'Kxd7', '{', '[%eval', '0.9]', '[%clk', '0:06:41]', '}', '8.', 'Be3', '{', '[%eval', '0.96]', '[%clk', '0:06:53]', '}', '8...', 'e6', '{', '[%eval', '1.15]',

In [3]:
import chess

In [4]:
move_list = []
looking_at = 0
move_ = None
for i in line.split():
    if looking_at == 0:
        looking_at = 1
    elif looking_at == 1:
        move_ = i
        while move_[-1] in ['!', '?']:
            move_ = move_[:-1]
        looking_at = 2
    elif looking_at == 2:
        looking_at = 3
    elif looking_at == 3:
        looking_at  = 4
    elif looking_at == 4:
        if i[0] == '#':
            break
        evaluation  = float(i[:-1])
        move_list.append((move_, evaluation))
        looking_at = 5
    elif looking_at == 7:
        looking_at = 0
    elif looking_at >= 5:
        looking_at += 1
print(move_list)


[('e4', 0.36), ('c5', 0.32), ('Nf3', 0.0), ('Nc6', 0.0), ('d4', 0.33), ('d6', 1.41), ('dxc5', 0.95), ('dxc5', 1.31), ('Qxd8+', 1.93), ('Nxd8', 1.67), ('Bb5+', 1.57), ('Bd7', 1.66), ('Bxd7+', 0.98), ('Kxd7', 0.9), ('Be3', 0.96), ('e6', 1.15), ('Ne5+', 1.35), ('Kd6', 1.96), ('Nxf7+', -2.9), ('Nxf7', -2.91), ('O-O', -3.34), ('Rc8', -3.24), ('Nd2', -3.42), ('Kc7', -3.3), ('Nb3', -3.61), ('b6', -3.53), ('c4', -4.21), ('Kb8', -3.87), ('Rfd1', -4.18), ('Ne5', 2.09), ('Bf4', 2.03), ('b5', 4.63), ('Bxe5+', 4.46), ('Ka8', 4.75), ('cxb5', 4.5), ('Nf6', 4.66), ('Bxf6', 4.44), ('gxf6', 4.6), ('Na5', 4.61), ('f5', 4.95), ('e5', 4.59), ('Bg7', 4.78), ('f4', 5.14), ('Rcd8', 5.1), ('Nc6', 5.24), ('Rxd1+', 5.4), ('Rxd1', 5.44), ('Rc8', 10.55), ('Rd7', 10.33)]


In [5]:
import chess
import torch

def board_to_tensor(board):
    tensor = torch.zeros(6, 8, 8)
    for file in range(8):
        for rank in range(8):
            square = chess.square(file, rank)
            piece = board.piece_at(square)
            if piece:
                piecet = piece.piece_type
                number = 1 if piece.color == chess.WHITE else -1
                tensor[int(piecet)-1, file, rank] = number
    return tensor

INIT_BOARD = board_to_tensor(chess.Board())

class TensorBoard:
    def __init__(self):
        self.board = chess.Board()
        self.tensor = INIT_BOARD.clone()

    def push(self, move: chess.Move):
        # Get the piece type and color
        from_square = move.from_square
        to_square = move.to_square
        piece = self.board.piece_at(from_square)
        piecet = piece.piece_type
        color = 1 if piece.color == chess.WHITE else -1

        # Clear the 'from' square
        from_file, from_rank = chess.square_file(from_square), chess.square_rank(from_square)
        self.tensor[:, from_file, from_rank] = 0

        # Set the 'to' square
        to_file, to_rank = chess.square_file(to_square), chess.square_rank(to_square)
        self.tensor[:, to_file, to_rank] = 0  # Clear any captured piece
        self.tensor[int(piecet)-1, to_file, to_rank] = color

        # Handle special moves
        if self.board.is_castling(move):
            if to_file > from_file:  # Kingside castling
                rook_from, rook_to = chess.H1 if color == 1 else chess.H8, chess.F1 if color == 1 else chess.F8
            else:  # Queenside castling
                rook_from, rook_to = chess.A1 if color == 1 else chess.A8, chess.D1 if color == 1 else chess.D8
            rook_from_file, rook_from_rank = chess.square_file(rook_from), chess.square_rank(rook_from)
            rook_to_file, rook_to_rank = chess.square_file(rook_to), chess.square_rank(rook_to)
            self.tensor[int(chess.ROOK)-1, rook_from_file, rook_from_rank] = 0
            self.tensor[int(chess.ROOK)-1, rook_to_file, rook_to_rank] = color
        elif self.board.is_en_passant(move):
            captured_pawn = chess.square(to_file, from_rank)
            captured_file, captured_rank = chess.square_file(captured_pawn), chess.square_rank(captured_pawn)
            self.tensor[int(chess.PAWN)-1, captured_file, captured_rank] = 0
        elif move.promotion:
            self.tensor[int(piecet)-1, to_file, to_rank] = 0
            self.tensor[int(move.promotion)-1, to_file, to_rank] = color

        # Update the underlying chess board
        self.board.push(move)


    def pop(self):
        move = self.board.pop()
        
        # Reverse the move
        from_square = move.to_square
        to_square = move.from_square
        piece = self.board.piece_at(to_square)
        piecet = piece.piece_type
        color = 1 if piece.color == chess.WHITE else -1

        # Clear the 'from' square (which was the 'to' square in the original move)
        from_file, from_rank = chess.square_file(from_square), chess.square_rank(from_square)
        self.tensor[:, from_file, from_rank] = 0

        # Set the 'to' square (which was the 'from' square in the original move)
        to_file, to_rank = chess.square_file(to_square), chess.square_rank(to_square)
        self.tensor[int(piecet)-1, to_file, to_rank] = color

        # Handle special moves
        if self.board.is_castling(move):
            if from_file > to_file:  # Kingside castling
                rook_to, rook_from = chess.H1 if color == 1 else chess.H8, chess.F1 if color == 1 else chess.F8
            else:  # Queenside castling
                rook_to, rook_from = chess.A1 if color == 1 else chess.A8, chess.D1 if color == 1 else chess.D8
            rook_to_file, rook_to_rank = chess.square_file(rook_to), chess.square_rank(rook_to)
            rook_from_file, rook_from_rank = chess.square_file(rook_from), chess.square_rank(rook_from)
            self.tensor[int(chess.ROOK)-1, rook_to_file, rook_to_rank] = 0
            self.tensor[int(chess.ROOK)-1, rook_from_file, rook_from_rank] = color
        elif self.board.is_en_passant(move):
            captured_pawn = chess.square(from_file, to_rank)
            captured_file, captured_rank = chess.square_file(captured_pawn), chess.square_rank(captured_pawn)
            self.tensor[int(chess.PAWN)-1, captured_file, captured_rank] = -color
        elif move.promotion:
            self.tensor[int(move.promotion)-1, from_file, from_rank] = 0
            self.tensor[int(chess.PAWN)-1, to_file, to_rank] = color

        # If a piece was captured, restore it
        captured_piece = self.board.piece_at(from_square)
        if captured_piece:
            captured_piecet = captured_piece.piece_type
            captured_color = 1 if captured_piece.color == chess.WHITE else -1
            self.tensor[int(captured_piecet)-1, from_file, from_rank] = captured_color

        return move

In [6]:
# Make the tensor into 12 64-bit integers
def Encode(tb : TensorBoard, evaluation : float):
    encoded = ''
    tnr = tb.tensor
    if tb.board.turn == chess.BLACK:
        tnr = -tnr
        evaluation = -evaluation
    for i in range(6):
        for sign in [-1, 1]:
            int_ = 0
            for j in range(64):
                if tnr[i, j//8, j%8] == sign:
                    int_ = 2*int_ + 1
                else:
                    int_ = 2*int_
            encoded += str(int_) + ' '
    
    return encoded+str(int(100*evaluation))

def decode(encoded_str: str):
    parts = encoded_str.split()
    evaluation = int(parts[-1])
    tensor = torch.zeros((6, 8, 8), dtype=torch.int)

    def decode_layer(encoded_value, sign):
        decoded_layer = torch.zeros((8, 8), dtype=torch.int)
        for j in range(64):
            if encoded_value & (1 << (63 - j)):
                decoded_layer[j // 8, j % 8] = sign
        return decoded_layer

    for i in range(6):
        for sign in [-1, 1]:
            encoded_value = int(parts[2 * i + (sign == -1)])
            tensor[i] += decode_layer(encoded_value, sign)

    return tensor, evaluation

In [7]:
saved_dataset = open("../Dataset/Processed/Dataset_V1.txt", 'w')

for line in Dataset:
    move_list = []
    looking_at = 0
    move_ = None
    brk = False
    for i in line.split():
        try:
            if looking_at == 0:
                looking_at = 1
            elif looking_at == 1:
                move_ = i
                while move_[-1] in ['!', '?']:
                    move_ = move_[:-1]
                looking_at = 2
            elif looking_at == 2:
                looking_at = 3
            elif looking_at == 3:
                looking_at  = 4
            elif looking_at == 4:
                if i[0] == '#':
                    break
                evaluation  = float(i[:-1])
                move_list.append((move_, evaluation))
                looking_at = 5
            elif looking_at == 7:
                looking_at = 0
            elif looking_at >= 5:
                looking_at += 1
        except:
            brk = True
            break
    if brk:
        continue

    TB = TensorBoard()
    try:
        for i, move in enumerate(move_list):
            if i<10:
                # Opening
                TB.push(TB.board.parse_san(move[0]))
                continue
            TB.push(TB.board.parse_san(move[0]))
            evaluation = move[1]

            # Save the tensor
            encoded = Encode(TB, evaluation)
            saved_dataset.write(encoded + '\n')
    except:
        continue

saved_dataset.close()

In [ ]:
for i in [2]:
    for sign in [-1, 1]:
        int_ = 0
        print(TB.tensor[i])
        for j in range(64):
            if TB.tensor[i, j//8, j%8] == sign:
                int_ = 2*int_ + 1
            else:
                int_ = 2*int_
        print(int_)
        print(TB.board)

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
0
r . . . k . n r
p p p . . p p p
. . . . . . . .
. . . P N . . .
. . B . . . . N
. . P . . . . .
P P . . . R P P
R . B Q . . K .
tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
158329674399744
r . . . k . n r
p p p . . p p p
. . . . . . . .
. . . P N . . .
. . B . . . . N
. . P . . . . .
P P . . . R P P
R . B Q . . K .


In [ ]:
# Make the tensor into 12 64-bit integers
def Encode(tb : TensorBoard, evaluation : float):
    encoded = ''
    tnr = tb.tensor
    if tb.board.turn == chess.BLACK:
        tnr = -tnr
    for i in range(6):
        for sign in [-1, 1]:
            int_ = 0
            for j in range(64):
                if tnr[i, j//8, j%8] == sign:
                    int_ = 2*int_ + 1
                else:
                    int_ = 2*int_
            encoded += str(int_) + ' '
    
    return encoded+str(int(100*evaluation))

def decode(encoded_str: str):
    parts = encoded_str.split()
    evaluation = int(parts[-1])
    tensor = torch.zeros((6, 8, 8), dtype=torch.int)

    def decode_layer(encoded_value, sign):
        decoded_layer = torch.zeros((8, 8), dtype=torch.int)
        for j in range(64):
            if encoded_value & (1 << (63 - j)):
                decoded_layer[j // 8, j % 8] = sign
        return decoded_layer

    for i in range(6):
        for sign in [-1, 1]:
            encoded_value = int(parts[2 * i + (sign == -1)])
            tensor[i] += decode_layer(encoded_value, sign)

    return tensor, evaluation

print(Encode(TB, 2.34))
print(decode(Encode(TB, 2.34)))

if not torch.equal(TB.tensor, decode(Encode(TB, 2.34))[0]):
    print('Error')

4629735635668713536 144680337052664322 134217744 256 158329674399744 0 9223372036858970112 72057594037927937 549755813888 0 32768 16777216 234
(tensor([[[ 0,  1,  0,  0,  0,  0, -1,  0],
         [ 0,  1,  0,  0,  0,  0, -1,  0],
         [ 0,  0,  1,  0,  0,  0, -1,  0],
         [ 0,  0,  0,  0,  1,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0, -1,  0],
         [ 0,  1,  0,  0,  0,  0, -1,  0],
         [ 0,  1,  0,  0,  0,  0, -1,  0]],

        [[ 0,  0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  1,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0, -1],
         [ 0,  0,  0,  1,  0,  0,  0,  0]],

        [[ 0,  0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0,  0],
         [ 1,  0,  0,  1,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  